# YK CASE 


In [276]:
import openpyxl
import xlsxwriter
import pandas as pd 
import numpy as np
from sklearn.cluster import KMeans
from scipy.stats import kstest
import plotly.express as px
import plotly.graph_objects as go
import openpyxl
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
import time 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier,XGBRegressor
from lightgbm import LGBMClassifier,LGBMRegressor
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
import shap
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from sklearn.preprocessing import OrdinalEncoder
from catboost import CatBoostClassifier

SEED = 42
warnings.filterwarnings("ignore")



In [277]:
#dfs = pd.read_excel("dosya.xlsx", sheet_name=None) 
data = pd.read_csv('./income_case_study.csv', delimiter=";")
data.drop_duplicates(inplace=True)
# Test datasının saklanması
val = data[data['income'].isna()]
data = data[data['income'].notnull()]


# Data Ön İşleme

In [278]:
data["education-num"] = data.groupby("education_level")["education-num"].transform(lambda x: x.fillna(x.max()))
data["income"] = data["income"].str.strip()
data["income"] = data["income"].map({'>50K': 1, '<=50K': 0})


In [279]:
# Satırlardaki boşlukları düzeltme
data["education_level"] = data["education_level"].str.strip()
data["workclass"] = data["workclass"].str.strip()
data["marital-status"] = data["marital-status"].str.strip()
data["occupation"] = data["occupation"].str.strip()
data["education_level"] = data["education_level"].str.strip()
data["relationship"] = data["relationship"].str.strip()
data["race"] = data["race"].str.strip()
data["sex"] = data["sex"].str.strip()
data["native-country"] = data["native-country"].str.strip()
#data["income"] = data["income"].str.strip()

In [280]:
data['education_level'].replace('Preschool', 'dropout',inplace=True)
data['education_level'].replace('10th', 'dropout',inplace=True)
data['education_level'].replace('11th', 'dropout',inplace=True)
data['education_level'].replace('12th', 'dropout',inplace=True)
data['education_level'].replace('1st-4th', 'dropout',inplace=True)
data['education_level'].replace('5th-6th', 'dropout',inplace=True)
data['education_level'].replace('7th-8th', 'dropout',inplace=True)
data['education_level'].replace('9th', 'dropout',inplace=True)
data['education_level'].replace('HS-Grad', 'HighGrad',inplace=True)
data['education_level'].replace('HS-grad', 'HighGrad',inplace=True)
data['education_level'].replace('Some-college', 'CommunityCollege',inplace=True)
data['education_level'].replace('Assoc-acdm', 'CommunityCollege',inplace=True)
data['education_level'].replace('Assoc-voc', 'CommunityCollege',inplace=True)
data['education_level'].replace('Bachelors', 'Bachelors',inplace=True)
data['education_level'].replace('Masters', 'Masters',inplace=True)
data['education_level'].replace('Prof-school', 'Masters',inplace=True)
data['education_level'].replace('Doctorate', 'Doctorate',inplace=True)

In [281]:
#df2 = my_df['marital-status'].replace(' Never-married', 'NotMarried')
data['marital-status'].replace('Never-married', 'NotMarried',inplace=True)
data['marital-status'].replace(['Married-AF-spouse'], 'Married',inplace=True)
data['marital-status'].replace(['Married-civ-spouse'], 'Married',inplace=True)
data['marital-status'].replace(['Married-spouse-absent'], 'NotMarried',inplace=True)
data['marital-status'].replace(['Separated'], 'Separated',inplace=True)
data['marital-status'].replace(['Divorced'], 'Separated',inplace=True)
data['marital-status'].replace(['Widowed'], 'Widowed',inplace=True)

In [282]:
data['occupation_flag'] = np.where(
    data['occupation'].isin(['Exec-managerial', 'Prof-specialty','Craft-repair']), # 'Craft-repair',
    1,
    0
)


In [283]:
data['net_capital'] = data['capital-gain'] - data['capital-loss']
data['has_capital_gain'] = data['capital-gain'].apply(lambda x: 1 if x > 0 else 0)

data['is_college_grad'] = data['education-num'].apply(lambda x: 1 if x >= 10 else 0)

def map_hours_to_type(hours):
    if hours <= 20:
        return "Part-time"
    elif hours <= 40:
        return "Full-time"
    else:
        return "Overwork"

data['hours_type'] = data['hours-per-week'].apply(map_hours_to_type)
data['is_American'] = data['native-country'].map(lambda x: 'United-States' if x == 'United-States' else 'Other')
data["not_paying"] = np.where(data["workclass"] == "Without-pay", 1, 0)

In [284]:
x = data[['age',
          'workclass', 'education_level',
       'marital-status',
          'occupation', 
          'race', 'sex',
      # 'capital-gain', 
          'capital-loss', 
         'hours-per-week','net_capital', 'has_capital_gain', 'is_American', "not_paying", "occupation_flag",'hours_type',
          'is_college_grad' 
         ]]
y = data[['income']]

# Test-Train ayrımı

In [285]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=SEED, stratify=y, 
    shuffle=True )

In [286]:
median_age = x_train['age'].median()
x_train['age'].fillna(median_age, inplace=True)

In [287]:
x_train['age-hours'] = x_train['age']*x_train['hours-per-week']

In [288]:
import pandas as pd

def get_education_mode_mapping(df):
    """
    Eğitim setinden occupation bazında en sık geçen education_level değerlerini çıkarır.
    """
    return df.groupby("occupation")["education_level"].agg(lambda x: x.mode()[0] if not x.mode().empty else None).to_dict()

def fill_missing_education(df, occupation_to_education):
    """
    Eksik education_level değerlerini occupation'a göre doldurur.
    """
    df["education_level"] = df.apply(
        lambda row: occupation_to_education.get(row["occupation"], "HighGrad") if pd.isna(row["education_level"]) else row["education_level"],
        axis=1
    )
    return df

# 1️⃣ Eğitim setinde occupation bazlı mode değerlerini hesapla
occupation_to_education = get_education_mode_mapping(x_train)

# 2️⃣ Eğitim setindeki eksik değerleri doldur
x_train = fill_missing_education(x_train, occupation_to_education)

# Kategorik Değişkenleri Etiketleme


In [289]:
# Kategorik sıralama listesi (hiyerarşik encoding için)
education_order = ["dropout", "HighGrad", "CommunityCollege", "Bachelors", "Masters", "Doctorate"]

# Veriyi temizleme ve küçük harfe çevirme (birebir eşleşme için)
x_train["education_level"] = x_train["education_level"].astype(str).str.strip()

# Ordinal Encoder oluşturma ve bilinmeyen değerleri -1 ile işaretleme
ordinal_encoder = OrdinalEncoder(categories=[education_order], handle_unknown="use_encoded_value", unknown_value=-1)

# Encoding işlemini gerçekleştir
x_train["education_level_encoded"] = ordinal_encoder.fit_transform(x_train[["education_level"]])


In [290]:
x_train['education_work'] = x_train["education_level_encoded"]*x_train['hours-per-week'] 

In [291]:
# One-Hot Encoding uygulama
x_train = pd.get_dummies(x_train, columns=[ "sex",  'marital-status',
                                           "race",
                                           "occupation","workclass",
                                           'has_capital_gain', 'is_American',"not_paying", "occupation_flag",'hours_type',
                                           'is_college_grad'
                                          ], drop_first=True)


In [292]:
x_train = x_train.drop(columns=["education_level"])

# Test datasını hazırlama 

In [293]:
x_test['age'].fillna(median_age, inplace=True)    
x_test['age-hours'] = x_test['age']*x_test['hours-per-week']
x_test = fill_missing_education(x_test, occupation_to_education)


In [294]:
x_test["education_level_encoded"] = ordinal_encoder.fit_transform(x_test[["education_level"]])
x_test['education_work'] = x_test["education_level_encoded"]*x_test['hours-per-week'] 

x_test = pd.get_dummies(x_test, columns=[ "sex", 'marital-status',
                                         "race",
                                         "occupation","workclass",
                                         'has_capital_gain','is_American',"not_paying", "occupation_flag",'hours_type',
                                         'is_college_grad'
                                         
                                        ], drop_first=True)

In [295]:
x_test = x_test.drop(columns=["education_level"])


# Model Kurulumu

In [296]:
from xgboost import XGBClassifier

# scale_pos_weight = (sınıf_0 sayısı) / (sınıf_1 sayısı)
scale_pos = len(y_train[y_train==0]) / len(y_train[y_train==1])

xgb_model = XGBClassifier(scale_pos_weight=scale_pos, random_state=42)
xgb_model.fit(x_train, y_train)
y_pred = xgb_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print("Test Seti Doğruluk Skoru:", accuracy)
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred))
f1 = f1_score(y_test, y_pred)
f1

Test Seti Doğruluk Skoru: 0.8645805592543275

Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.89      0.93      0.91      5407
           1       0.79      0.70      0.74      2103

    accuracy                           0.86      7510
   macro avg       0.84      0.81      0.83      7510
weighted avg       0.86      0.86      0.86      7510



0.74324665488513